In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteria, StoppingCriteriaList

class StopTokenCriteria(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # 检查最新生成的 token 是否是停止标记
        return input_ids[0, -1] in self.stop_token_id

# Load the tokenizer and model
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#tokenizer = AutoTokenizer.from_pretrained("/home/dozhang/nlcmt/HuggingfaceModels/Llama-2-13b-chat-hf")
#model = AutoModelForCausalLM.from_pretrained("/home/dozhang/nlcmt/HuggingfaceModels/Llama-2-13b-chat-hf", torch_dtype=torch.bfloat16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained("/home/dozhang/nlcmt2/HuggingfaceModels/Meta-Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("/home/dozhang/nlcmt2/HuggingfaceModels/Meta-Llama-3.1-8B-Instruct", torch_dtype=torch.bfloat16, device_map="auto")

#tokenizer = AutoTokenizer.from_pretrained("/home/dozhang/nlcmt1/HuggingfaceModels/Meta-Llama-3.1-70B-Instruct")
#model = AutoModelForCausalLM.from_pretrained("/home/dozhang/nlcmt1/HuggingfaceModels/Meta-Llama-3.1-70B-Instruct", torch_dtype=torch.bfloat16, device_map="auto")

/home/dozhang/miniconda3/envs/py31/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-10-15 02:30:02,345] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/dozhang/miniconda3/envs/py31/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/dozhang/miniconda3/envs/py31/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/dozhang/miniconda3/envs/py31/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/dozhang/miniconda3/envs/py31/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/dozhang/miniconda3/envs/py31/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/dozhang/miniconda3/envs/py31/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/dozhang/miniconda3/envs/py31/com

In [2]:


input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n2+5=10, 3+6=18, 4+7=?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

input_text = "Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?"


input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nif let 2+3=13, 3+4=25, 4+5=41, 5+6=?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant for travel tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Kelian has two recipes for preparing dishes, one having 20 instructions and the second one having twice as many instructions as the first one. How many instructions does Kelian have to read to prepare the two dishes?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Create attention mask
attention_mask = inputs.attention_mask

# 定义停止条件
stop_token_id = tokenizer.convert_tokens_to_ids(["</s>", "<|eot_id|>", "<|end_of_text|>", "<|end_header_id|>", "<|start_header_id|>"])
stopping_criteria = StoppingCriteriaList([StopTokenCriteria(stop_token_id)])


In [3]:
from hf_model_adapt import hf_adapt

### Todo: complete hf_adapt 
#(model, sum1, sum2, sum3, over_zero, flat_zero) = hf_adapt(model, tokenizer, nBarLayer=54, valBarSim=0.96, nOutLayer = 4, nCheckLayer=2, nWarmupTok = -1)
#(model, sum1, sum2, sum3, over_zero, flat_zero) = hf_adapt(model, tokenizer, nBarLayer=24, valBarSim=0.96, nOutLayer = 4, nCheckLayer=2, nWarmupTok = -1)
(model, globalNumDecodedLayer, globalNumSkippedLayer, sum3, over_zero, flat_zero) = hf_adapt(model, tokenizer, nBarLayer=20, valBarSim=0.93, nOutLayer = 3, nCheckLayer=2, nWarmupTok = -1)

#import pdb; pdb.set_trace()
# Generate text
generation_kwargs = {
                        "do_sample":False,
                         "temperature":0, 
                         "top_p":1
                    }
outputs = model.generate(inputs["input_ids"], attention_mask=attention_mask, max_length=512, num_return_sequences=1, 
                         pad_token_id=tokenizer.eos_token_id, stopping_criteria=stopping_criteria, **generation_kwargs)

# Decode the generated text
input_length = inputs["input_ids"].shape[1]
generated_text = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=False)

print(f'\nPrompt::: {input_text}\n')
print(f'Response >>> {generated_text}')

/home/dozhang/miniconda3/envs/py31/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



*** Set BarLayer=20 based on prompt-layer similairty over 71 tokens.

--- No truncation at #position 70, #SimScore 0.9609375, for token ['<|begin_of_text|>', '<|begin_of_text|>', '<|start_header_id|>', 'system', '<|end_header_id|>', 'ĊĊ', 'You', 'Ġare', 'Ġa', 'Ġhelpful', 'ĠAI', 'Ġassistant', 'Ġfor', 'Ġtravel', 'Ġtips', 'Ġand', 'Ġrecommendations', '<|eot_id|>', '<|start_header_id|>', 'user', '<|end_header_id|>', 'ĊĊ', 'Question', ':', 'ĠKel', 'ian', 'Ġhas', 'Ġtwo', 'Ġrecipes', 'Ġfor', 'Ġpreparing', 'Ġdishes', ',', 'Ġone', 'Ġhaving', 'Ġ', '20', 'Ġinstructions', 'Ġand', 'Ġthe', 'Ġsecond', 'Ġone', 'Ġhaving', 'Ġtwice', 'Ġas', 'Ġmany', 'Ġinstructions', 'Ġas', 'Ġthe', 'Ġfirst', 'Ġone', '.', 'ĠHow', 'Ġmany', 'Ġinstructions', 'Ġdoes', 'ĠKel', 'ian', 'Ġhave', 'Ġto', 'Ġread', 'Ġto', 'Ġprepare', 'Ġthe', 'Ġtwo', 'Ġdishes', '?', '<|eot_id|>', '<|start_header_id|>', 'assistant', '<|end_header_id|>']

--- No truncation at #position 71, #SimScore 0.9296875, for token ['ĊĊ']

--- No truncation at #posi